In [1]:
import pandas as pd
from pathlib import Path

In [2]:
routes = pd.read_csv('routes.dat', sep='\t', usecols=['airline','A','B']).drop_duplicates()
airports = pd.read_csv('airport-codes.csv', sep=',')[['iata_code', 'iso_country', 'coordinates']]
airports = pd.concat([airports.drop(columns='coordinates'), airports.coordinates.str.split(', ', expand=True)], axis=1)
airports = airports[~airports.iata_code.isnull()].rename(columns={0:'lon',1:'lat', 'iata_code':'code', 'iso_country':'country'})

def save_df(airlines, name, coloring):
    df = routes[routes.airline.isin(airlines)]
    df = df.merge(airports, how='left', left_on='A', right_on='code').merge(airports, how='left', left_on='B', right_on='code', suffixes=('_A','_B'))
    df = df.drop(columns=['code_A','code_B'])
    df['color'] = df.apply(coloring, axis=1)
    df['AB'] = df.apply(lambda row: f"{row['A']}_{row['B']}", axis=1)

    df = df[(df.country_A=='US') & (df.country_B=='US')]

    df = df.sample(frac=1)
    
    p = Path(name)
    p.mkdir(exist_ok=True)
    df.to_csv(f'{name}/routes.csv', index=False)
    return df

# All

In [3]:
%%time
def coloring(row):
    if row.airline == 'AA': return '#6cff7d'
    if row.airline == 'DL': return '#ff0000'
    if row.airline == 'UA': return '#426ae1'

save_df(['AA', 'DL', 'UA'], 'all', coloring)

CPU times: user 102 ms, sys: 5.32 ms, total: 107 ms
Wall time: 107 ms


,airline,A,B,country_A,lon_A,lat_A,country_B,lon_B,lat_B,color,AB
1288,DL,DTW,CID,US,-83.35340118408203,42.212398529052734,US,-91.71080017089844,41.884700775146484,#ff0000,DTW_CID
388,AA,DFW,PIA,US,-97.038002,32.896801,US,-89.6932983398,40.664199829100006,#6cff7d,DFW_PIA
91,AA,CLT,MCI,US,-80.94309997558594,35.2140007019043,US,-94.713898,39.2976,#6cff7d,CLT_MCI
2292,UA,IAH,MIA,US,-95.34140014648438,29.984399795532227,US,-80.29060363769531,25.79319953918457,#426ae1,IAH_MIA
1033,DL,ATL,ECP,US,-84.428101,33.6367,US,-85.795414,30.357106,#ff0000,ATL_ECP
...,...,...,...,...,...,...,...,...,...,...,...
1882,UA,DEN,HDN,US,-104.672996521,39.861698150635,US,-107.2180023,40.48120117,#426ae1,DEN_HDN
2140,UA,IAD,CVG,US,-77.45580292,38.94449997,US,-84.667801,39.048801,#426ae1,IAD_CVG
552,AA,LGA,ORF,US,-73.872597,40.777199,US,-76.20120239257812,36.89459991455078,#6cff7d,LGA_ORF
1194,DL,BOS,FLL,US,-71.00520325,42.36429977,US,-80.152702,26.072599,#ff0000,BOS_FLL


# American

In [4]:
%%time
def coloring(row):
    hubs = ['CLT','ORD','DFW','LAX','MIA','LGA','PHL', 'PHX', 'DCA']
    colors = ['#ff0000', '#ffff00', '#ffa500', '#00ffff', '#6cff7d', '#426ae1', '#fd68b3', '#426ae1', '#6cff7d']

    if row['A'] in hubs and row['B'] in hubs: return 'white'
    for k,v in zip(hubs, colors):
        if k in row.values: return v
    return 'gray'

save_df(['AA'], 'american', coloring)

CPU times: user 80.5 ms, sys: 4.38 ms, total: 84.9 ms
Wall time: 96 ms


,airline,A,B,country_A,lon_A,lat_A,country_B,lon_B,lat_B,color,AB
81,AA,CLT,JFK,US,-80.94309997558594,35.2140007019043,US,-73.7789,40.639801,#ff0000,CLT_JFK
815,AA,PHL,CHO,US,-75.24109649658203,39.87189865112305,US,-78.4529037475586,38.13859939575195,#fd68b3,PHL_CHO
407,AA,DFW,SAV,US,-97.038002,32.896801,US,-81.20210266,32.12760162,#ffa500,DFW_SAV
778,AA,ORD,SMF,US,-87.9048,41.9786,US,-121.59100341796875,38.69540023803711,#ffff00,ORD_SMF
226,AA,DCA,SYR,US,-77.037697,38.8521,US,-76.1063003540039,43.11119842529297,#6cff7d,DCA_SYR
...,...,...,...,...,...,...,...,...,...,...,...
221,AA,DCA,RSW,US,-77.037697,38.8521,US,-81.75520324707031,26.53619956970215,#6cff7d,DCA_RSW
173,AA,DCA,CAK,US,-77.037697,38.8521,US,-81.44219970703125,40.916099548339844,#6cff7d,DCA_CAK
584,AA,MIA,CHS,US,-80.29060363769531,25.79319953918457,US,-80.04049683,32.89860153,#6cff7d,MIA_CHS
239,AA,DFW,ABI,US,-97.038002,32.896801,US,-99.68190002440001,32.4113006592,#ffa500,DFW_ABI


# Delta

In [5]:

def coloring(row):
    hubs = ['ATL','BOS','DTW','JFK','LAX','LGA','MSP','SEA','SLC']
    colors = ['#ff0000', '#ffff00', '#ffa500', '#00ffff', '#007f00', '#ffddad', '#fd68b3', '#426ae1', '#6cff7d']

    if row['A'] in hubs and row['B'] in hubs: return 'white'
    for k,v in zip(hubs, colors):
        if k in row.values: return v
    return 'gray'

save_df(['DL'], 'delta', coloring)

,airline,A,B,country_A,lon_A,lat_A,country_B,lon_B,lat_B,color,AB
508,DL,LAX,CMH,US,-118.407997,33.942501,US,-82.891899,39.998001,#007f00,LAX_CMH
131,DL,ATL,MLB,US,-84.428101,33.6367,US,-80.64530181884766,28.102800369262695,#ff0000,ATL_MLB
112,DL,ATL,LGA,US,-84.428101,33.6367,US,-73.872597,40.777199,white,ATL_LGA
809,DL,SLC,ORD,US,-111.97799682617188,40.78839874267578,US,-87.9048,41.9786,#6cff7d,SLC_ORD
323,DL,DTW,DAY,US,-83.35340118408203,42.212398529052734,US,-84.21939849853516,39.902400970458984,#ffa500,DTW_DAY
...,...,...,...,...,...,...,...,...,...,...,...
31,DL,ATL,BTV,US,-84.428101,33.6367,US,-73.15329742429999,44.471900939899996,#ff0000,ATL_BTV
297,DL,DCA,MSN,US,-77.037697,38.8521,US,-89.3375015258789,43.13990020751953,gray,DCA_MSN
351,DL,DTW,LAS,US,-83.35340118408203,42.212398529052734,US,-115.1520004,36.08010101,#ffa500,DTW_LAS
689,DL,MSP,SLC,US,-93.221802,44.882,US,-111.97799682617188,40.78839874267578,white,MSP_SLC


# United

In [6]:

def coloring(row):
    hubs = ['ORD','DEN','IAH','LAX','EWR','SFO','IAD']
    colors = ['#ff0000', '#ffff00', '#ffa500', '#00ffff', '#6cff7d', '#426ae1', '#fd68b3', '#426ae1', '#6cff7d']

    if row['A'] in hubs and row['B'] in hubs: return 'white'
    for k,v in zip(hubs, colors):
        if k in row.values: return v
    return 'gray'

save_df(['UA'], 'united', coloring)

,airline,A,B,country_A,lon_A,lat_A,country_B,lon_B,lat_B,color,AB
741,UA,ORD,STL,US,-87.9048,41.9786,US,-90.370003,38.748697,#ff0000,ORD_STL
317,UA,IAD,BOS,US,-77.45580292,38.94449997,US,-71.00520325,42.36429977,#fd68b3,IAD_BOS
167,UA,DEN,XWA,US,-104.672996521,39.861698150635,US,-103.748797,48.258387,#ffff00,DEN_XWA
563,UA,LAX,COS,US,-118.407997,33.942501,US,-104.700996,38.805801,#00ffff,LAX_COS
325,UA,IAD,CKB,US,-77.45580292,38.94449997,US,-80.2281036377,39.2966003418,#fd68b3,IAD_CKB
...,...,...,...,...,...,...,...,...,...,...,...
262,UA,EWR,ROC,US,-74.168701171875,40.692501068115234,US,-77.67240142822266,43.118900299072266,#6cff7d,EWR_ROC
457,UA,IAH,ICT,US,-95.34140014648438,29.984399795532227,US,-97.433098,37.649899,#ffa500,IAH_ICT
740,UA,ORD,SRQ,US,-87.9048,41.9786,US,-82.55439758300781,27.39539909362793,#ff0000,ORD_SRQ
540,UA,IAH,TPA,US,-95.34140014648438,29.984399795532227,US,-82.533203125,27.975500106811523,#ffa500,IAH_TPA
